# Make bound and free states from the prepped inputs

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/projects/crispy_shifty/notebooks
running on node: dig99


### Set working directory to the root of the crispy_shifty repo

In [2]:
os.chdir("/projects/crispy_shifty")

### Make helix-bound states from the scaffolds

In [6]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "02_make_bound_states"
design_list_file = os.path.join(
    os.getcwd(),
    "scaffolds/01_prep_inputs/prepped_inputs.list",
)
output_path = os.path.join(os.getcwd(), f"scaffolds/{simulation_name}")

options = " ".join(
    [
        "out:level 200",
    ]
)

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.states.make_bound_states",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="medium",
    memory="2G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    simulation_name=simulation_name,
)

Run the following command with your desired environment active:
sbatch -a 1-32266 /mnt/projects/crispy_shifty/scaffolds/02_make_bound_states/run.sh


### Make free states from the scaffolds

In [8]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "02_make_free_states"
design_list_file = os.path.join(
    os.getcwd(),
    "scaffolds/01_prep_inputs/prepped_inputs.list",
)
output_path = os.path.join(os.getcwd(), f"scaffolds/{simulation_name}")

options = " ".join(
    [
        "out:level 200",
    ]
)

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.states.make_free_states",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="medium",
    memory="3G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    simulation_name=simulation_name,
)

Run the following command with your desired environment active:
sbatch -a 1-32266 /mnt/projects/crispy_shifty/scaffolds/02_make_free_states/run.sh


### Collect scorefiles of the bound and free states

In [3]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "02_make_bound_states"
output_path = os.path.join(os.getcwd(), f"scaffolds/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

simulation_name = "02_make_free_states"
output_path = os.path.join(os.getcwd(), f"scaffolds/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


### Load resulting scorefiles of bound and free states

In [4]:
sys.path.insert(0, os.getcwd())
from crispy_shifty.utils.io import parse_scorefile_linear

bound_scores_df = parse_scorefile_linear(
    os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/scores.json")
)

free_scores_df = parse_scorefile_linear(
    os.path.join(os.getcwd(), "scaffolds/02_make_free_states/scores.json")
)

  0%|          | 0/550533 [00:00<?, ?it/s]

  0%|          | 0/663475 [00:00<?, ?it/s]

### Dump scorefiles as CSVs and then reload, for performance reasons

In [3]:
if not os.path.exists(
    os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/scores.csv")
):
    bound_scores_df.to_csv(
        os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/scores.csv")
    )

bound_scores_df = pd.read_csv(
    os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/scores.csv"),
    index_col="Unnamed: 0",
)

if not os.path.exists(
    os.path.join(os.getcwd(), "scaffolds/02_make_free_states/scores.csv")
):
    free_scores_df.to_csv(
        os.path.join(os.getcwd(), "scaffolds/02_make_free_states/scores.csv")
    )

free_scores_df = pd.read_csv(
    os.path.join(os.getcwd(), "scaffolds/02_make_free_states/scores.csv"),
    index_col="Unnamed: 0",
)

### Save a list of outputs

In [4]:
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/bound_states.list"), "w"
) as f:
    for path in tqdm(bound_scores_df.index):
        print(path, file=f)
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_free_states/free_states.list"), "w"
) as f:
    for path in tqdm(free_scores_df.index):
        print(path, file=f)

  0%|          | 0/550533 [00:00<?, ?it/s]

  0%|          | 0/663475 [00:00<?, ?it/s]

### Save also a CSV of just free states that have 0 shift
We will need them later

In [6]:
output_path = os.path.join(
    os.getcwd(), "scaffolds/02_make_free_states/free_state_0s.csv"
)
free_state_0s = free_scores_df.query("shift == 0 and pivot_helix == pre_break_helix")
free_state_0s.to_csv(output_path)

### Save lists of only bound JHRs, DHRs, junctions etc.

In [9]:
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/JHRs.list"), "w"
) as f:
    for path in tqdm(
        bound_scores_df.query(
            "scaffold_type == 'bcov_JHR' or scaffold_type == 'drhicks1_JHR'"
        ).index
    ):
        print(path, file=f)
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/DHRs.list"), "w"
) as f:
    for path in tqdm(
        bound_scores_df.query(
            "scaffold_type == 'tj_DHRs_filtered' or scaffold_type == 'tj_DHRs_final'"
        ).index
    ):
        print(path, file=f)
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/junctions.list"), "w"
) as f:
    for path in tqdm(
        bound_scores_df.query(
            "scaffold_type == 'tj_junctions' or scaffold_type == 'tj_junctions_l1'"
        ).index
    ):
        print(path, file=f)
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/non_junctions.list"), "w"
) as f:
    for path in tqdm(
        bound_scores_df.query(
            "scaffold_type != 'tj_junctions' and scaffold_type != 'tj_junctions_l1'"
        ).index
    ):
        print(path, file=f)

  0%|          | 0/42651 [00:00<?, ?it/s]

  0%|          | 0/1990 [00:00<?, ?it/s]

  0%|          | 0/505892 [00:00<?, ?it/s]

  0%|          | 0/44641 [00:00<?, ?it/s]

### Save lists of only free JHRs, DHRs, junctions etc.

In [11]:
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_free_states/JHRs.list"), "w"
) as f:
    for path in tqdm(
        free_scores_df.query(
            "scaffold_type == 'bcov_JHR' or scaffold_type == 'drhicks1_JHR'"
        ).index
    ):
        print(path, file=f)
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_free_states/DHRs.list"), "w"
) as f:
    for path in tqdm(
        free_scores_df.query(
            "scaffold_type == 'tj_DHRs_filtered' or scaffold_type == 'tj_DHRs_final'"
        ).index
    ):
        print(path, file=f)
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_free_states/junctions.list"), "w"
) as f:
    for path in tqdm(
        free_scores_df.query(
            "scaffold_type == 'tj_junctions' or scaffold_type == 'tj_junctions_l1'"
        ).index
    ):
        print(path, file=f)
with open(
    os.path.join(os.getcwd(), "scaffolds/02_make_free_states/non_junctions.list"), "w"
) as f:
    for path in tqdm(
        free_scores_df.query(
            "scaffold_type != 'tj_junctions' and scaffold_type != 'tj_junctions_l1'"
        ).index
    ):
        print(path, file=f)

  0%|          | 0/52971 [00:00<?, ?it/s]

  0%|          | 0/2156 [00:00<?, ?it/s]

  0%|          | 0/608348 [00:00<?, ?it/s]

  0%|          | 0/55127 [00:00<?, ?it/s]

### Prototyping blocks

test `make_bound_states`

In [ ]:
%%time 
import pyrosetta

pyrosetta.init()


sys.path.insert(0, "/projects/crispy_shifty/")
from crispy_shifty.protocols.states import make_bound_states
# from crispy_shifty.protocols.states import make_bound_states


t = make_bound_states(
        None,
        **{
            'pdb_path': '/mnt/projects/crispy_shifty/scaffolds/01_prep_inputs/decoys/0000/notebooks_01_prep_inputs_fa1b5ca9cef5486383f1054118203438.pdb.bz2',
            'name': 'DHR78_DHR71_l2_0_v2c',
            'pre_break_helix': 2,
#             'clash_cutoff': 5000,
#             'int_cutoff': 0.9,
#             'full_helix': True,
        }
)

In [ ]:
for i, tppose in enumerate(t):
    tppose.pose.dump_pdb(f"{tppose.scores['state']}.pdb")

test `grow_terminal_helices`

In [ ]:
import pyrosetta

sys.path.insert(0, "/projects/crispy_shifty/")
from crispy_shifty.protocols.states import grow_terminal_helices


pyrosetta.init()
tpose = pyrosetta.pose_from_file(
    "/home/pleung/projects/bistable_bundle/r4/helix_binders/08_analysis/pdbs/cs_088_Y.pdb"
)
tpose2 = grow_terminal_helices(
    pose=tpose,
    chain=2,
    extend_n_term=7,
    extend_c_term=7,
)

test `extend_helix_termini`

In [ ]:
import pyrosetta

sys.path.insert(0, "/projects/crispy_shifty/")
from crispy_shifty.protocols.states import extend_helix_termini


pyrosetta.init()
tpose = pyrosetta.pose_from_file(
    "/home/pleung/projects/bistable_bundle/r4/helix_binders/08_analysis/pdbs/cs_088_Y.pdb"
)
tpose2 = extend_helix_termini(
    pose=tpose,
    chain=2,
    extend_n_term=7,
    extend_c_term=7,
)

In [ ]:
tpose2.dump_pdb("test2.pdb")